# Copyright 2017 Google Inc.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Imports

In [1]:
import tensorflow as tf
import numpy as np
from pprint import pprint as show

print(tf.__version__)

2023-08-06 11:07:53.281861: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-06 11:07:53.282962: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-06 11:07:53.306074: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-06 11:07:53.306604: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-06 11:07:53.794746: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not fin

2.13.0


## Data set
From https://en.wikipedia.org/wiki/Iris_flower_data_set

3 types of Iris Flowers: 

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/5/56/Kosaciec_szczecinkowaty_Iris_setosa.jpg/450px-Kosaciec_szczecinkowaty_Iris_setosa.jpg" style="width: 100px; display:inline"/>
<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/4/41/Iris_versicolor_3.jpg/800px-Iris_versicolor_3.jpg" style="width: 150px;display:inline"/>
<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/9/9f/Iris_virginica.jpg/736px-Iris_virginica.jpg" style="width: 150px;display:inline"/>
* Iris Setosa
* Iris Versicolour
* Iris Virginica




## Data Columns:
   1. sepal length in cm 
   2. sepal width in cm 
   3. petal length in cm 
   4. petal width in cm

<img src="petal_sepal.png" style="width: 200px;"/>
<img src="https://storage.googleapis.com/image-uploader/AIA_images/data_table.png" style="width: 450px"/>

## Set creation function

In [2]:
def make_set(dataset):
    data = []
    labels = []
    for dt, label in dataset.as_numpy_iterator():
        data += dt.tolist()
        for lbl in label.tolist():
            for l in lbl:
                labels.append(int(l))

    return {
        'data': data,
        'target': labels
    }

## Load data in

In [3]:
# in TensorFlow 2 contrib is DEPRECATED
#from tensorflow.contrib.learn.python.learn.datasets import base
import tensorflow_datasets as base

# Data files
# IRIS_TRAINING = "iris_training.csv"
# IRIS_TEST = "iris_test.csv"
DATA = 'iris'

# Load datasets - for TF v1.x
# training_set = base.load_csv_with_header(filename=IRIS_TRAINING,
#                                          features_dtype=np.float32,
#                                          target_dtype=np.int)
# test_set = base.load_csv_with_header(filename=IRIS_TEST,
#                                      features_dtype=np.float32,
#                                      target_dtype=np.int)
# print(training_set.data)
# print(training_set.target)

# Load datasets - for TF v2.x
training_set = base.load(DATA,split='train[:80%]', as_supervised=True)
training_set = training_set.batch(10).map(lambda x, y: (x, tf.one_hot(y, depth=1)))
training_set = make_set(training_set)
test_set = base.load(DATA,split='train[:20%]', as_supervised=True)
test_set = test_set.batch(10).map(lambda x, y: (x, tf.one_hot(y, depth=1)))
test_set = make_set(test_set)

show(training_set['data'])
show(training_set['target'])


/usr/local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[[5.099999904632568, 3.4000000953674316, 1.5, 0.20000000298023224],
 [7.699999809265137, 3.0, 6.099999904632568, 2.299999952316284],
 [5.699999809265137, 2.799999952316284, 4.5, 1.2999999523162842],
 [6.800000190734863, 3.200000047683716, 5.900000095367432, 2.299999952316284],
 [5.199999809265137,
  3.4000000953674316,
  1.399999976158142,
  0.20000000298023224],
 [5.599999904632568,
  2.9000000953674316,
  3.5999999046325684,
  1.2999999523162842],
 [5.5, 2.5999999046325684, 4.400000095367432, 1.2000000476837158],
 [5.5, 2.4000000953674316, 3.700000047683716, 1.0],
 [4.599999904632568,
  3.4000000953674316,
  1.399999976158142,
  0.30000001192092896],
 [7.699999809265137, 2.799999952316284, 6.699999809265137, 2.0],
 [7.0, 3.200000047683716, 4.699999809265137, 1.399999976158142],
 [4.599999904632568, 3.200000047683716, 1.399999976158142, 0.20000000298023224],
 [6.5, 3.0, 5.199999809265137, 2.0],
 [5.5, 4.199999809265137, 1.399999976158142, 0.20000000298023224],
 [5.400000095367432,
  3

## Feature columns and model creation

In [4]:
# Specify that all features have real-value data
feature_name = "flower_features"
feature_columns = [tf.feature_column.numeric_column(feature_name, 
                                                    shape=[4])]

classifier = tf.estimator.LinearClassifier(
    feature_columns=feature_columns,
    n_classes=3,
    model_dir="/tmp/iris_model")


Instructions for updating:
Use Keras preprocessing layers instead, either directly or via the `tf.keras.utils.FeatureSpace` utility. Each of `tf.feature_column.*` has a functional equivalent in `tf.keras.layers` for feature preprocessing when training a Keras model.


Instructions for updating:
Use Keras preprocessing layers instead, either directly or via the `tf.keras.utils.FeatureSpace` utility. Each of `tf.feature_column.*` has a functional equivalent in `tf.keras.layers` for feature preprocessing when training a Keras model.


Instructions for updating:
Use tf.keras instead.


Instructions for updating:
Use tf.keras instead.


Instructions for updating:
Use tf.keras instead.


...

Instructions for updating:
Use tf.keras instead.


INFO:tensorflow:Using default config.


INFO:tensorflow:Using default config.


INFO:tensorflow:Using config: {'_model_dir': '/tmp/iris_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Using config: {'_model_dir': '/tmp/iris_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


## Input function

In [5]:
def input_fn(dataset):
    def _fn():
        features = {feature_name: tf.constant(dataset['data'])}
        label = tf.constant(dataset['target'])
        return features, label
    return _fn

print(input_fn(training_set)())

# raw data -> input function -> feature columns -> model

({'flower_features': <tf.Tensor: shape=(120, 4), dtype=float32, numpy=
array([[5.1, 3.4, 1.5, 0.2],
       [7.7, 3. , 6.1, 2.3],
       [5.7, 2.8, 4.5, 1.3],
       [6.8, 3.2, 5.9, 2.3],
       [5.2, 3.4, 1.4, 0.2],
       [5.6, 2.9, 3.6, 1.3],
       [5.5, 2.6, 4.4, 1.2],
       [5.5, 2.4, 3.7, 1. ],
       [4.6, 3.4, 1.4, 0.3],
       [7.7, 2.8, 6.7, 2. ],
       [7. , 3.2, 4.7, 1.4],
       [4.6, 3.2, 1.4, 0.2],
       [6.5, 3. , 5.2, 2. ],
       [5.5, 4.2, 1.4, 0.2],
       [5.4, 3.9, 1.3, 0.4],
       [5. , 3.5, 1.3, 0.3],
       [5.1, 3.8, 1.5, 0.3],
       [4.8, 3. , 1.4, 0.1],
       [6.5, 3. , 5.8, 2.2],
       [7.6, 3. , 6.6, 2.1],
       [6.7, 3.3, 5.7, 2.1],
       [7.9, 3.8, 6.4, 2. ],
       [6.7, 3. , 5.2, 2.3],
       [5.8, 4. , 1.2, 0.2],
       [6.3, 2.5, 5. , 1.9],
       [5. , 3. , 1.6, 0.2],
       [6.9, 3.1, 5.1, 2.3],
       [6.1, 3. , 4.6, 1.4],
       [5.8, 2.7, 4.1, 1. ],
       [5.2, 2.7, 3.9, 1.4],
       [6.7, 3. , 5. , 1.7],
       [5.7, 2.6, 3.5, 1. ],
 

## Training

In [6]:
# Fit model.
classifier.train(input_fn=input_fn(training_set),
               steps=1000)
print('fit done')


Instructions for updating:
Use tf.keras instead.


Instructions for updating:
Use tf.keras instead.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


...

INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 13000...


INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 13000...


INFO:tensorflow:Loss for final step: 0.00029951028.


INFO:tensorflow:Loss for final step: 0.00029951028.


fit done


## Evaluation

In [7]:
# Evaluate accuracy.
accuracy_score = classifier.evaluate(input_fn=input_fn(test_set), 
                                     steps=100)["accuracy"]
print('\nAccuracy: {0:f}'.format(accuracy_score))

INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Done calling model_fn.


INFO:tensorflow:Done calling model_fn.


INFO:tensorflow:Starting evaluation at 2023-08-06T11:08:18


...

INFO:tensorflow:Saving dict for global step 13000: accuracy = 1.0, average_loss = 0.00016007299, global_step = 13000, loss = 0.00016007286


INFO:tensorflow:Saving dict for global step 13000: accuracy = 1.0, average_loss = 0.00016007299, global_step = 13000, loss = 0.00016007286


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 13000: /tmp/iris_model/model.ckpt-13000


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 13000: /tmp/iris_model/model.ckpt-13000



Accuracy: 1.000000


## Exporting a model for serving predictions


In [8]:
# Tensorflow 1.x
#feature_spec = {'flower_features': tf.FixedLenFeature(shape=[4], dtype=np.float32)}
# Tensorflow 2.x
feature_spec = {'flower_features': tf.io.FixedLenFeature(shape=[4], dtype=np.float32)}
serving_fn = tf.estimator.export.build_parsing_serving_input_receiver_fn(feature_spec)

# Tensorflow 1.x
# classifier.export_savedmodel(export_dir_base='/tmp/iris_model' + '/export', 
#                             serving_input_receiver_fn=serving_fn)
# Tensorflow 2.x
classifier.export_saved_model(export_dir_base='/tmp/iris_model' + '/export', 
                            serving_input_receiver_fn=serving_fn)

Instructions for updating:
Use tf.keras instead.


Instructions for updating:
Use tf.keras instead.


Instructions for updating:
Use tf.keras instead.


Instructions for updating:
Use tf.keras instead.


INFO:tensorflow:Calling model_fn.


...

INFO:tensorflow:No assets to write.


INFO:tensorflow:No assets to write.


INFO:tensorflow:SavedModel written to: /tmp/iris_model/export/temp-1691320121/saved_model.pb


INFO:tensorflow:SavedModel written to: /tmp/iris_model/export/temp-1691320121/saved_model.pb


b'/tmp/iris_model/export/1691320121'